<a href="https://colab.research.google.com/github/Alby-Benny-IBM/PySpark/blob/main/05_ScalaUseCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,773 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [2]:
!apt install -y mysql-server mysql-client


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl
  libfcgi-bin libfcgi-perl libfcgi0ldbl libhtml-parser-perl
  libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2
  libprotobuf-lite23 liburi-perl mecab-ipadic mecab-ipadic-utf8 mecab-utils
  mysql-client-8.0 mysql-client-core-8.0 mysql-server-8.0
  mysql-server-core-8.0
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libbusiness-isbn-perl libwww-perl
  mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl
  libfcgi-bin libfcgi-perl libfcgi0ldbl libhtml-parser-perl
  libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2
  lib

In [3]:
!service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [13]:
%%bash
# Install required packages for connecting to database

# Download MySQL JDBC connector
wget https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-j-8.2.0.tar.gz
tar -xzf mysql-connector-j-8.2.0.tar.gz
cp mysql-connector-j-8.2.0/mysql-connector-j-8.2.0.jar /content/


--2025-08-04 07:02:14--  https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-j-8.2.0.tar.gz
Resolving dev.mysql.com (dev.mysql.com)... 23.223.133.87, 2600:1408:c400:1881::2e31, 2600:1408:c400:188c::2e31
Connecting to dev.mysql.com (dev.mysql.com)|23.223.133.87|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cdn.mysql.com//archives/mysql-connector-java-8.2/mysql-connector-j-8.2.0.tar.gz [following]
--2025-08-04 07:02:15--  https://cdn.mysql.com//archives/mysql-connector-java-8.2/mysql-connector-j-8.2.0.tar.gz
Resolving cdn.mysql.com (cdn.mysql.com)... 23.223.131.232, 2600:1408:c400:1884::1d68, 2600:1408:c400:188d::1d68
Connecting to cdn.mysql.com (cdn.mysql.com)|23.223.131.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4243603 (4.0M) [application/x-tar-gz]
Saving to: ‘mysql-connector-j-8.2.0.tar.gz’

     0K .......... .......... .......... .......... ..........  1% 3.05M 1s
    50K .......... 

In [ ]:
import os
os.kill(os.getpid(), 9)


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import base64, col

In [2]:
spark = SparkSession.builder \
    .appName("MySQL Connection") \
    .config("spark.jars", "/content/mysql-connector-j-8.2.0.jar") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print("Spark session created successfully!")
print(f"Spark version: {spark.version}")

Spark session created successfully!
Spark version: 3.5.1


In [3]:
jdbc_url = "jdbc:mysql://localhost:3306/classicmodels"
connection_properties = {
    "user": "lucifer",
    "password": "lucifer",
    "driver": "com.mysql.cj.jdbc.Driver"
}

print("Connection properties configured!")

Connection properties configured!


In [13]:
tables = [
    "productlines",
    "products",
    "offices",
    "employees",
    "customers",
    "payments",
    "orders",
    "orderdetails"
]

# Output folders
csv_folder = "/content/output_csv"
json_folder = "/content/output_json"

for table in tables:
    print(f"Processing table: {table}")
    df = spark.read.jdbc(url=jdbc_url, table=table, properties=connection_properties)

    # Identify and encode binary columns
    converted_columns = []
    for col_name, col_type in df.dtypes:
        if col_type == "binary":
            converted_columns.append(base64(col(col_name)).alias(col_name))
        else:
            converted_columns.append(col(col_name))

    df_converted = df.select(converted_columns)

    # Write to CSV folder
    df_converted.write.mode("overwrite") \
        .option("header", "true") \
        .csv(f"{csv_folder}/{table}")

    # Write to JSON folder
    df_converted.write.mode("overwrite") \
        .json(f"{json_folder}/{table}")

print("✅ Export completed. All CSVs saved to 'output_csv/', JSONs to 'output_json/'")

Processing table: productlines
Processing table: products
Processing table: offices
Processing table: employees
Processing table: customers
Processing table: payments
Processing table: orders
Processing table: orderdetails
✅ Export completed. All CSVs saved to 'output_csv/', JSONs to 'output_json/'


In [20]:
!ls -R /content/output_json/

/content/output_json/:
customers  offices	 orders    productlines
employees  orderdetails  payments  products

/content/output_json/customers:
part-00000-85c5c95e-1587-44c4-a5f5-6ec7d642801e-c000.json  _SUCCESS

/content/output_json/employees:
part-00000-909037bc-669f-4e8b-b11a-00d75bfed707-c000.json  _SUCCESS

/content/output_json/offices:
part-00000-011ca56f-23f3-4190-8498-9eb8b5ba407d-c000.json  _SUCCESS

/content/output_json/orderdetails:
part-00000-6ce2dabc-b9a9-488d-b3e0-9adb8fe560c4-c000.json  _SUCCESS

/content/output_json/orders:
part-00000-692f3313-9962-44ba-a1e8-17e8ac6d8d47-c000.json  _SUCCESS

/content/output_json/payments:
part-00000-881b5b2a-9383-4bc5-8176-647ebc741d74-c000.json  _SUCCESS

/content/output_json/productlines:
part-00000-0271de90-f7d7-459a-96a7-3b67a3b4c800-c000.json  _SUCCESS

/content/output_json/products:
part-00000-ddd24075-55ef-4028-b1af-298b9968ebf3-c000.json  _SUCCESS


In [21]:
import os

def clean_and_rename(base_dir, ext):
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            files = os.listdir(folder_path)
            for file_name in files:
                file_path = os.path.join(folder_path, file_name)
                # Delete non-target-extension files
                if not file_name.endswith(ext):
                    os.remove(file_path)
                else:
                    # Rename target file to folder_name.ext
                    new_file_path = os.path.join(folder_path, f"{folder_name}{ext}")
                    os.rename(file_path, new_file_path)

# Paths and extensions
csv_dir = '/content/output_csv/'
json_dir = '/content/output_json/'

clean_and_rename(csv_dir, '.csv')
clean_and_rename(json_dir, '.json')
